## Testing Localozation net

In [ ]:
from torchvision.transforms import Normalize
from localization.datasets import DepthDataset, ColorDataset, ColorDepthDataset
from utils.arguments import Arguments

import torch
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150
import matplotlib.pyplot as plt
import numpy as np
from localization.localization import MappingVAE
import cv2


args = Arguments.get_arguments()

rgb_dataset = ColorDataset(args.data_path, '00', hls=False)

rgb_normalization = Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
hls_normalization = Normalize(mean=(70.2209, 62.9458), std=(47.7068, 63.5109))
depth_normalization = Normalize(mean=(0.4855), std=(0.169))

known = (torch.randint(0, len(rgb_dataset)//5, (1, 1))*5).item()
sample = known+3
print(known)
print(sample)

In [ ]:
known = (torch.randint(0, len(rgb_dataset)//5, (1, 1))*5).item()
sample = known+3
print(known)

model = MappingVAE(input_type='RGB')
model.load_state_dict(torch.load("checkpoints/localization_RGB.pth", map_location='cpu'))
model.eval()
image = rgb_dataset[sample]
plt.imshow(image.permute(1, 2, 0).byte()); plt.show()
_, _, _, reconstruction = model(image.unsqueeze(0))

reconstruction = reconstruction.squeeze()

# RGB
mean=torch.tensor((0.485, 0.456, 0.406)).view(3, 1, 1)
std=torch.tensor((0.229, 0.224, 0.225)).view(3, 1, 1)

#y = (reconstruction*std)+mean
y = reconstruction

mi = torch.amin(y, dim=(1, 2)).view(3, 1, 1)
y = y-mi
ma = torch.amax(y, dim=(1, 2)).view(3, 1, 1)
y = y/ma

plt.imshow(y.permute(1, 2, 0).detach()); plt.show()

In [ ]:
test_code, encoded, normed_code = model.get_code(rgb_dataset[sample].unsqueeze(0))
test_code = test_code.squeeze().to('cpu').detach()

code = test_code.flatten(1)
def min_max_hist(code):
    print(code.shape)
    #mini = torch.amin(code, -1).int().detach().numpy()
    #maxi = torch.amax(code, -1).int().detach().numpy()
    #plt.hist(mini, np.arange(np.min(mini), np.max(mini), 10)); plt.title("Min"); plt.show()
    #plt.hist(maxi, np.arange(np.min(maxi), np.max(maxi), 10)); plt.title("Max"); plt.show()

print(encoded.shape)
print(test_code.shape)
print(normed_code.shape)
min_max_hist(encoded.squeeze().flatten(1))
min_max_hist(test_code.squeeze().flatten(1))
min_max_hist(normed_code.squeeze().flatten(1))

code = code.view(16, 8, 4, 13)

row_delimiter = torch.ones((1, 113)).float()
col_delimiter = torch.ones((4, 1)).float()

plane = []
plane.append(row_delimiter)
for y in range(16):
    row = []
    row.append(col_delimiter)
    for x in range(8):
        row.append(code[y, x])
        row.append(col_delimiter)
    row = torch.cat(row, dim=1)
    plane.append(row)
    plane.append(row_delimiter)

plane = torch.cat(plane, dim=0)

plt.imshow(plane, cmap="gray"); plt.show()
print(plane.shape)